In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
tf.enable_eager_execution()

In [3]:
ds = tf.data.TFRecordDataset('/data/invoices_train.tfrecord')

In [4]:
for element in ds.take(1):
    print(element)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [5]:
def parse_tfrecords(tensor_string):
    """ 
    Decode a serialized tensor
    That takes a image_dicts normalize and returns a tuple (x,y)
    Returns a x(image) to decode
    """
    feature_decode = {
        'image_raw': tf.io.FixedLenFeature([], tf.string, default_value=''),
        'label': tf.io.FixedLenFeature([], tf.int64, default_value=0)}

    img_dic = tf.io.parse_single_example(tensor_string, feature_decode)
    
    label = img_dic['label']
    image = tf.io.parse_tensor(serialized=img_dic["image_raw"], out_type=tf.uint8)

    return {'image': image, 'label': label}

In [6]:
ds1 = ds.map(parse_tfrecords)
ds1

<DatasetV1Adapter shapes: {image: <unknown>, label: ()}, types: {image: tf.uint8, label: tf.int64}>

In [18]:
for element in ds1.take(1):
    print(tf.image.resize_images(element['image'], [224, 224]))

tf.Tensor(
[[[255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]
  ...
  [255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]]

 [[255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]
  ...
  [255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]]

 [[255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]
  ...
  [255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]]

 ...

 [[255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]
  ...
  [255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]]

 [[255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]
  ...
  [255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]]

 [[255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]
  ...
  [255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]]], shape=(224, 224, 3), dtype=float32)


In [37]:
def preprocess_data(image_dic):
    """  
    Function that takes a image_dic, normalized (divided by 255) and returns a tupe (x, y)
    """

    image, label = image_dic['image'], image_dic['label']
    
    image.set_shape([None, None, None])
    #image = tf.image.decode_jpeg(image)
    image = tf.math.divide(image, 255)
    image = tf.image.resize_images(image, (224, 224))
    image = tf.reshape(image, [224*224*3])

    return (image, label)

ds2 = ds1.map(preprocess_data)
ds2

<DatasetV1Adapter shapes: ((150528,), ()), types: (tf.float32, tf.int64)>

In [38]:
for element in ds2.take(1):
    print(element)

(<tf.Tensor: id=259, shape=(150528,), dtype=float32, numpy=array([1., 1., 1., ..., 1., 1., 1.], dtype=float32)>, <tf.Tensor: id=260, shape=(), dtype=int64, numpy=67>)


In [ ]:
inputs = tf.keras.Input(shape=(224*224*3,), name='Hiraganas')
x = tf.keras.layers.Dense(64, activation='relu')(inputs)
x = tf.keras.layers.BatchNormalization(renorm=True)(x)
x = tf.keras.layers.Dense(32, activation='relu')(x)
x = tf.keras.layers.BatchNormalization(renorm=True)(x)
outputs = tf.keras.layers.Dense(10, activation='softmax')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)
# "sparse me sirve para clasificacion mutuamente excluyente un label --- multilabel"
model.compile(optimizer='Adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [ ]:
ds2 = ds2.shuffle(buffer_size=30000).batch(32)
ds2 = ds2.batch(64)
history = model.fit(ds2, epochs=3)